In [19]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import math
pd.set_option('display.max_columns', None)


In [20]:
data = pd.read_csv('feature_df.csv', index_col=0)
data = data.drop(columns=['Season', 'Week', 'Date', 'Time',
       'H/A', 'W/L', 'PF', 'PA'])

In [21]:
data

,COV,COV_ema16,COV_ema4,COV_ema8,COV_opp,COV_sma16,COV_sma4,COV_sma8,DEF,DEF_ema16,DEF_ema4,DEF_ema8,DEF_opp,DEF_sma16,DEF_sma4,DEF_sma8,OFF,OFF_ema16,OFF_ema4,OFF_ema8,OFF_opp,OFF_sma16,OFF_sma4,OFF_sma8,OVR,OVR_ema16,OVR_ema4,OVR_ema8,OVR_opp,OVR_sma16,OVR_sma4,OVR_sma8,Opposing Team,PASS,PASS_ema16,PASS_ema4,PASS_ema8,PASS_opp,PASS_sma16,PASS_sma4,PASS_sma8,PBLK,PBLK_ema16,PBLK_ema4,PBLK_ema8,PBLK_opp,PBLK_sma16,PBLK_sma4,PBLK_sma8,PRSH,PRSH_ema16,PRSH_ema4,PRSH_ema8,PRSH_opp,PRSH_sma16,PRSH_sma4,PRSH_sma8,RBLK,RBLK_ema16,RBLK_ema4,RBLK_ema8,RBLK_opp,RBLK_sma16,RBLK_sma4,RBLK_sma8,RDEF,RDEF_ema16,RDEF_ema4,RDEF_ema8,RDEF_opp,RDEF_sma16,RDEF_sma4,RDEF_sma8,RECV,RECV_ema16,RECV_ema4,RECV_ema8,RECV_opp,RECV_sma16,RECV_sma4,RECV_sma8,RUN,RUN_ema16,RUN_ema4,RUN_ema8,RUN_opp,RUN_sma16,RUN_sma4,RUN_sma8,TACK,TACK_ema16,TACK_ema4,TACK_ema8,TACK_opp,TACK_sma16,TACK_sma4,TACK_sma8,Team,gamenum
256,62.1,60.450180,54.809857,58.487722,91.4,61.98125,55.525,59.2875,54.1,66.966685,65.599649,66.365232,85.3,67.66250,64.300,64.6000,51.4,70.961263,66.960660,69.386184,92.5,72.20625,68.000,70.5875,51.0,71.366499,67.877750,69.961591,92.2,72.56250,68.000,70.1125,Indianapolis Colts,45.1,72.000468,66.911982,70.417042,94.4,72.34375,69.400,72.5125,61.7,75.508031,67.981033,72.903918,72.7,77.55000,72.575,74.8625,60.8,59.793764,56.224653,58.431659,64.4,61.01875,58.175,58.8125,72.3,66.869224,64.893794,65.789756,91.8,68.56875,67.275,64.6250,42.7,71.572692,75.313838,72.607822,61.2,71.50625,71.475,68.5500,48.6,66.899567,64.649521,65.623689,76.6,68.25625,62.575,66.4875,51.5,59.961106,60.571151,60.844634,59.9,58.03750,61.325,62.1750,78.4,61.920323,63.588733,62.118108,71.5,62.29375,58.975,60.6125,New Orleans Saints,256
256,91.4,68.014896,68.624754,68.120924,62.1,68.04375,69.000,67.6750,85.3,60.695354,60.191356,60.165949,54.1,61.05625,54.850,60.6375,92.5,77.941612,76.070044,77.005941,51.4,78.91250,72.900,77.4875,92.2,73.090921,71.051888,72.028456,51.0,73.97500,66.375,73.0375,New Orleans Saints,94.4,79.675368,78.306386,79.038721,45.1,80.51250,75.175,78.6625,72.7,80.784854,81.182233,81.023853,61.7,80.53750,84.525,80.9500,64.4,63.620907,59.146699,61.600594,60.8,65.58750,58.100,64.1750,91.8,67.671836,63.944302,65.845502,72.3,69.68750,62.625,69.0125,61.2,54.143010,54.074189,54.230310,42.7,53.15625,49.375,55.0000,76.6,74.063251,75.100205,74.114985,48.6,74.51250,71.275,71.3375,59.9,60.989418,57.965217,60.353890,51.5,60.13750,55.650,65.4250,71.5,60.757450,60.938291,60.365051,78.4,60.90625,57.900,61.8000,Indianapolis Colts,256
257,72.6,69.437507,69.791714,69.980452,54.4,68.61875,69.875,69.8125,65.8,69.260170,72.295877,70.502452,55.8,68.28125,71.300,69.2750,83.4,64.574687,64.198218,64.035675,67.1,65.67500,64.925,61.8375,80.2,67.882996,69.301968,68.222496,61.6,68.00625,69.200,66.2125,Buffalo Bills,68.2,56.377816,59.661136,56.977108,45.9,57.08750,60.450,51.8250,74.0,77.403229,84.218893,79.427334,91.2,76.25625,83.775,74.7375,55.0,60.476308,65.682665,62.154048,56.8,59.25625,62.675,61.0875,88.2,65.230776,67.625521,65.926175,68.5,65.28750,66.950,64.2000,60.5,63.683653,64.098932,63.401674,57.0,64.61875,61.200,63.0500,76.5,66.424017,63.923403,65.334923,63.0,67.40000,66.150,64.5875,68.9,53.646190,48.762963,51.626601,69.1,55.33125,47.300,53.5375,43.4,64.221561,64.287881,65.028520,62.4,62.06875,64.475,68.5375,Denver Broncos,257
257,54.4,73.625385,72.722614,74.259768,72.6,71.52500,77.275,77.4000,55.8,68.746427,68.056881,69.324353,65.8,66.85000,73.050,70.4000,67.1,61.331329,57.967864,60.331254,83.4,62.20625,61.575,62.4125,61.6,66.171960,63.594637,65.861273,80.2,65.65000,69.325,67.6750,Denver Broncos,45.9,52.427992,44.804683,49.879083,68.2,54.69375,54.550,54.1875,91.2,74.729231,71.469910,74.522711,74.0,73.13125,73.425,76.9750,56.8,58.807003,53.747583,56.622278,55.0,61.10000,56.000,56.9500,68.5,59.331143,55.331051,57.659785,88.2,61.30625,56.975,58.8875,57.0,60.995842,64.240307,62.980593,60.5,58.15000,68.600,60.5625,63.0,68.373534,68.765394,68.675131,76.5,68.34375,70.900,68.9750,69.1,52.

In [ ]:
%%time
# feature_importances_
pp = pprint.PrettyPrinter(indent=4)
for attribute in attributes:
    # Lists to hold on to the generated values
    gbr_preds = []
    bagr_preds = []
    t = []
    teams = []
    games = []
    
    columns = []
    
    if attribute == 'OVR':
        # lags
        for i in range(1, 17):
            columns.append('OVR_lag' + str(i))
            columns.append('OFF_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            columns.append('RECV_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('OVR_avg' + str(j))
            # opposing
            columns.append('OVR_avg' + str(j) + '_opposing')
            columns.append('DEF_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')
        
    elif attribute == 'OFF':
        #lags
        for i in range(1, 17):
            columns.append('OFF_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            columns.append('RECV_lag' + str(i))
            columns.append('PBLK_lag' + str(i))
            columns.append('RBLK_lag' + str(i))
            columns.append('RUN_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('OFF_avg' + str(j))
            columns.append('PASS_avg' + str(j))
            columns.append('RECV_avg' + str(j))
            columns.append('PBLK_avg' + str(j))
            columns.append('RBLK_avg' + str(j))
            columns.append('RUN_avg' + str(j))
            # opposing
            columns.append('DEF_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')
            columns.append('RDEF_avg' + str(j) + '_opposing')
            columns.append('PRSH_avg' + str(j) + '_opposing')

    elif attribute == 'DEF':
        # lags
        for i in range(1, 17):
            columns.append('DEF_lag' + str(i))
            columns.append('OVR_lag' + str(i))
            columns.append('PRSH_lag' + str(i))
            columns.append('RDEF_lag' + str(i))
            columns.append('COV_lag' + str(i))
            columns.append('TACK_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('DEF_avg' + str(j))
            columns.append('OVR_avg' + str(j))
            columns.append('PRSH_avg' + str(j))
            columns.append('RDEF_avg' + str(j))
            columns.append('COV_avg' + str(j))
            columns.append('TACK_avg' + str(j))
            # opposing
            columns.append('OFF_avg' + str(j) + '_opposing')
            columns.append('PASS_avg' + str(j) + '_opposing')
            columns.append('RBLK_avg' + str(j) + '_opposing')
            columns.append('RECV_avg' + str(j) + '_opposing')
            columns.append('PBLK_avg' + str(j) + '_opposing')

    elif attribute == 'RBLK':
        # lags
        for i in range(1, 17):
            columns.append('RBLK_lag' + str(i))
            columns.append('PBLK_lag' + str(i))
        
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('RBLK_avg' + str(j))
            columns.append('PBLK_avg' + str(j))
            # opposing
            columns.append('RDEF_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')
            columns.append('PRSH_avg' + str(j) + '_opposing')

    elif attribute == 'PBLK':
        # lags
        for i in range(1, 17):
            columns.append('PBLK_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('PBLK_avg' + str(j))
            # opposing
            columns.append('PRSH_avg' + str(j) + '_opposing')
    
    elif attribute == 'RECV':
        # lags
        for i in range(1, 17):
            columns.append('RECV_lag' + str(i))
            columns.append('PASS_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('RECV_avg' + str(j))
            columns.append('PASS_avg' + str(j))
            # opposing
            columns.append('TACK_avg' + str(j) + '_opposing')
            columns.append('PRSH_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')

    elif attribute == 'PASS':
        # lags
        for i in range(1, 17):
            columns.append('PASS_lag' + str(i))
            columns.append('RECV_lag' + str(i))
            columns.append('PBLK_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('PASS_avg' + str(j))
            columns.append('RECV_avg' + str(j))
            columns.append('PBLK_avg' + str(j))
            # opposing
            columns.append('PRSH_avg' + str(j) + '_opposing')
            columns.append('COV_avg' + str(j) + '_opposing')

    elif attribute == 'RUN':
        # lags
        for i in range(1, 17):
            columns.append('RBLK_lag' + str(i))
            columns.append('RUN_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('RBLK_avg' + str(j))
            columns.append('RUN_avg' + str(j))
            # opposing
            columns.append('TACK_avg' + str(j) + '_opposing')
            columns.append('RDEF_avg' + str(j) + '_opposing')

    elif attribute == 'TACK':
        # lags
        for i in range(1, 17):
            columns.append('TACK_lag' + str(i))
            columns.append('RDEF_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('TACK_avg' + str(j))
            columns.append('RDEF_avg' + str(j))
            # opposing
            columns.append('RECV_avg' + str(j) + '_opposing')
            columns.append('RUN_avg' + str(j) + '_opposing')

    elif attribute == 'RDEF':
        # lags
        for i in range(1, 17):
            columns.append('TACK_lag' + str(i))
            columns.append('RDEF_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('TACK_avg' + str(j))
            columns.append('RDEF_avg' + str(j))
            # opposing
            columns.append('RBLK_avg' + str(j) + '_opposing')
            columns.append('RUN_avg' + str(j) + '_opposing')

    elif attribute == 'PRSH':
        # lags
        for i in range(1, 17):
            columns.append('PRSH_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('PRSH_avg' + str(j))
            # opposing
            columns.append('PBLK_avg' + str(j) + '_opposing')
            columns.append('PASS_avg' + str(j) + '_opposing')
            columns.append('RECV_avg' + str(j) + '_opposing')

    elif attribute == 'COV':
        # lags
        for i in range(1, 17):
            columns.append('PRSH_lag' + str(i))
            columns.append('COV_lag' + str(i))
            columns.append('TACK_lag' + str(i))
            
        # averages
        for j in (2,3,4,6,8,12,16):
            columns.append('PRSH_avg' + str(j))
            columns.append('COV_avg' + str(j))
            columns.append('TACK_avg' + str(j))
            # opposing
            columns.append('PBLK_avg' + str(j) + '_opposing')
            columns.append('RECV_avg' + str(j) + '_opposing')
            columns.append('PASS_avg' + str(j) + '_opposing')

    for team in grouped:
        data = team[1]
        
        
        # Train test split
        train = data[data['Season'] < 2017]
        test = data[data['Season'] >= 2017]
        
        y_train = train[attribute]
        X_train = train[columns]
        
        y_test = test[attribute]
        X_test = test[columns]
        
        team = test['Team'].values
        gameNum = test['GameNum'].values
        
        
        # Gradient Boosting Regressor
        gbr = GradientBoostingRegressor()
        
        # Bagging Regressor
        bag = BaggingRegressor()
        
        # Train models
        gbr.fit(X_train, y_train)
        bag.fit(X_train, y_train)
        
        
        
        # Now Cross Validate
        grid_search = GridSearchCV(gbr, gbr_parameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
        grid_search.fit(X_train, y_train)
        
        print(attribute, "GBR", grid_search.best_params_)

        gbr_regressor = GradientBoostingRegressor(
            loss=grid_search.best_params_['loss'],
            max_depth=grid_search.best_params_['max_depth'],
            n_estimators=grid_search.best_params_['n_estimators'],
            learning_rate=grid_search.best_params_['learning_rate']
        )
        
        
        #Save best model parameters
        best_boost = best_boost.append({'Attribute' : attribute,
                                        'Team' : team[0],
                                        'loss' : grid_search.best_params_['loss'],
                                        'learning_rate' : grid_search.best_params_['learning_rate'],
                                        'max_depth' : grid_search.best_params_['max_depth'],
                                        'n_estimators' : grid_search.best_params_['n_estimators']},
                                      ignore_index = True)
        
        
        
        # CV BAG
        grid_search = GridSearchCV(bag, bag_parameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=3)
        grid_search.fit(X_train, y_train)
        
        print(attribute, "BAG", grid_search.best_params_)

        bag_regressor = BaggingRegressor(
            
            max_samples=grid_search.best_params_['max_samples'],
            n_estimators=grid_search.best_params_['n_estimators']
        )
        
        # Save the best model parameters
        best_bag = best_bag.append({'Attribute' : attribute,
                                    'Team' : team[0],
                                    'max_samples' : grid_search.best_params_['max_samples'],
                                    'n_estimators' : grid_search.best_params_['n_estimators']},
                                  ignore_index = True)
        
        
        # Train cross validated models
        gbr_regressor.fit(X_train, y_train)
        bag_regressor.fit(X_train, y_train)
        
        # feature_importance
        features = list(X_train.columns)
        gbr_regressor_importances = list(gbr_regressor.feature_importances_)
        gbr_regressor_feature_importance = dict.fromkeys(features, 0)
        i = 0
        for key in gbr_regressor_feature_importance.keys():
            gbr_regressor_feature_importance[key] = gbr_regressor_importances[i]
            i += 1
        variable_importances.append([attribute, team[0], gbr_regressor_feature_importance])
        
        # Make predictions
        gbr_hat = gbr_regressor.predict(X_test)
        bagr_hat = bag_regressor.predict(X_test)
        
        
        # Save each team's data
        for i in range(len(y_test)):
            t.append(y_test.iloc[i])
            gbr_preds.append(gbr_hat[i])
            bagr_preds.append(bagr_hat[i])
            teams.append(team[i])
            games.append(gameNum[i])
    
    # Save data for each attribute    
    gbr_predictions[attribute] = gbr_preds
    bagr_predictions[attribute] = bagr_preds
    target[attribute] = t
    
    gbr_predictions['Team'] = teams
    gbr_predictions['GameNum'] = games
    bagr_predictions['Team'] = teams
    bagr_predictions['GameNum'] = games
    target['Team'] = teams
    target['GameNum'] = games
    
    # Takes a while to run, so this shows progress
    print('\n\n\n\n\n\nFinished creating preditcions for:', attribute, '\n\n\n\n\n\n\n')